# 1. Logistic Regression
In this note, we will implement the logistic regression algorithm for binary classification. 

Let $(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), \ldots, (x^{(n)}, y^{(n)})$ be a sequence of training samples, where $x^{(i)} \in \mathbb{R}^d$, and $y^{(i)} \in \{0, 1\}$ for $i = 1, \ldots, n$. Let
$$X := \begin{pmatrix} x^{(1)^T} \\ \vdots \\  x^{(n)^T} \end{pmatrix}, y := \begin{pmatrix} y^{(1)} \\ \vdots \\  y^{(n)} \end{pmatrix}.$$

Our goal in binary classification is to train a model $h$ that will predict the label of a new sample $x$. We often use the following model:
$$h_{\theta}(x) = g(x^T \theta),$$
where
$$ \theta := \begin{pmatrix} \theta_{0} \\ \vdots \\  \theta_{d} \end{pmatrix}, $$ 
and
$$g(z) = \frac{1}{1 + e^{-z}}$$
is called the **logistic function** or the **sigmoid function**. We have also used the convention that $x^T \theta = \theta_0 + \sum_{i = 1}^{d}\theta_i x_i$. Note that $g \in (0, 1)$, which serves as a good choice as far as binary classification is concerned. In order to train our model, we try to maximize the log-likelihood function given as follows:
$$\ell(\theta) = \log L(\theta) = \sum_{i = 1}^{n}y^{(i)}\log(h_{\theta}(x^{(i)})) + (1 - y^{(i)})\log(1 - h_{\theta}(x^{(i)}))$$.

## 1.1 Gradient Ascent

There are several methods to maximize $\ell(\theta)$. We will discuss some of the more popular ones here. The first is by using the *gradient ascent* method. That is, our updates to $\theta$ will be given by $\theta \leftarrow \theta + \alpha \nabla_{\theta}\ell(\theta)$, where $\alpha$ is the so-called *learning rate*, and $\nabla$ denotes the gradient. By taking the gradient of $\ell(\theta)$ and using the fact that $g'(z) = g(z)(1 - g(z))$, we arrive at $\theta \leftarrow \theta - \alpha X^T (h_{\theta}(X) - y)$, where $h_{\theta}(X)$ is understood in the vector form. Below, you can find an implementation of the gradient ascent approach. 

In [16]:
import numpy as np
import matplotlib.pyplot as plt

In [19]:
class LogisticRegression:
    def __init__(self, learning_rate = 0.01, iterations = 1000):
        self.theta = None
        self.learning_rate = learning_rate
        self.iterations = iterations
    
    def fit(self, X, y):
        n_samples, d_features = X.shape
        X = np.hstack([np.ones((n_samples, 1)), X])
        self.theta = np.zeros(d_features + 1)
        alpha = self.learning_rate
        for k in range(self.iterations):
            z = X @ self.theta
            g = X.T @ (self._sigmoid(z) - y)
            self.theta = self.theta - alpha * g
        
        y_pred = self._sigmoid(z)
        cost = np.sum(np.log(y_pred)*y + np.log(1 - y_pred)*(1 - y))
        print(cost)
        return self.theta
            
        
    def predict(self, X):
        n_samples = X.shape[0]
        X = np.hstack([np.ones((n_samples, 1)), X])
        z = self._sigmoid(X @ self.theta)
        return np.round(z).astype(int)
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

In [20]:
# create sample dataset
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
y = np.array([0, 0, 1, 1, 1])

# initialize logistic regression model
LR = LogisticRegression()

# train model on sample dataset
LR.fit(X, y)

# make predictions on new data
X_new = np.array([[6, 7], [7, 8]])
y_pred = LR.predict(X_new)

print(y_pred)  # [1, 1]

-0.9048467015131948
[1 1]


## 1.2 Batch Gradient Ascent
One caveat of the gradient ascent or descent method in general (not specific to binary classification) is that we use all $n$ training samples to compute the gradient function. Naturally, this could be too costly if $n$ is very large. Another approach that bypasses this issue is the *batch gradient ascent* method, where we only use small batches of the training sample in every iteration in order to compute the gradient. Below, you can find an implementation of the batch gradient ascent approach. 

In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [25]:
class LogisticRegression:
    def __init__(self, batch_size = 32, learning_rate = 0.01, iterations = 1000):
        self.theta = None
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.iterations = iterations
    
    def fit(self, X, y):
        n_samples, d_features = X.shape
        X = np.hstack([np.ones((n_samples, 1)), X])
        self.theta = np.zeros(d_features + 1)
        alpha = self.learning_rate
        for k in range(self.iterations):
            batch_indices = np.random.choice(n_samples, self.batch_size)
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            z = X_batch @ self.theta
            g = X_batch.T @ (self._sigmoid(z) - y_batch)
            self.theta = self.theta - alpha * g
        
        y_pred = self._sigmoid(X @ self.theta)
        cost = np.sum(np.log(y_pred)*y + np.log(1 - y_pred)*(1 - y))
        print(cost)
        return self.theta
            
        
    def predict(self, X):
        n_samples = X.shape[0]
        X = np.hstack([np.ones((n_samples, 1)), X])
        z = self._sigmoid(X @ self.theta)
        return np.round(z).astype(int)
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

In [26]:
# create sample dataset
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
y = np.array([0, 0, 1, 1, 1])

# initialize logistic regression model
LR = LogisticRegression(batch_size = 2)

# train model on sample dataset
LR.fit(X, y)

# make predictions on new data
X_new = np.array([[6, 7], [7, 8]])
y_pred = LR.predict(X_new)

print(y_pred)  # [1, 1]

-1.419651602719068
[1 1]
